In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("myapp") \
        .master("local") \
        .config("spark.executor.memory", "1g") \
        .config("spark.mongodb.input.uri","mongodb://172.17.0.2:27017") \
        .config("spark.mongodb.output.uri","mongodb://172.17.0.2:27017") \
        .config("spark.jars.packages","org.mongodb.spark:mongo-spark-connector_2.12:3.0.0") \
        .getOrCreate()


In [2]:
conf = spark.sparkContext.getConf()
print("# spark.app.name = ", conf.get("spark.app.name"))
print("# spark.master = ", conf.get("spark.master"))
print("# spark.executor.memory = ", conf.get("spark.executor.memory"))
print("# spark.sql.warehouse.dir = ", conf.get("spark.sql.warehouse.dir"))
print("# spark.sql.catalogImplementation = ", conf.get("spark.sql.catalogImplementation"))

# spark.app.name =  myapp
# spark.master =  local
# spark.executor.memory =  1g
# spark.sql.warehouse.dir =  file:/home/jovyan/HiveMetastore/spark-warehouse
# spark.sql.catalogImplementation =  None


In [3]:
df = spark.read.format("mongo") \
               .option("database","test") \
               .option("collection","products") \
               .load()

In [4]:
df.write.mode("overwrite").save("products_new")

In [5]:
spark.sql("CREATE EXTERNAL TABLE external_products USING parquet LOCATION '/home/jovyan/HiveMetastore/products_new'")

DataFrame[]

In [6]:
spark.sql("DESCRIBE EXTENDED external_products").show(100,100)

+----------------------------+--------------------------------------------+-------+
|                    col_name|                                   data_type|comment|
+----------------------------+--------------------------------------------+-------+
|                   ListPrice|                                      double|   null|
|                    MakeFlag|                                         int|   null|
|                   ModelName|                                      string|   null|
|                   ProductID|                                         int|   null|
|                 ProductName|                                      string|   null|
|               ProductNumber|                                      string|   null|
|                StandardCost|                                      double|   null|
|               SubCategoryID|                                         int|   null|
|                         _id|                          struct<oid:string>| 

In [7]:
%ls -l metastore_db/

ls: cannot access 'metastore_db/': No such file or directory


In [8]:
%ls -l spark-warehouse

total 0


In [9]:
spark.sql("SELECT * FROM external_products").show()

+---------+--------+--------------------+---------+--------------------+-------------+------------+-------------+--------------------+
|ListPrice|MakeFlag|           ModelName|ProductID|         ProductName|ProductNumber|StandardCost|SubCategoryID|                 _id|
+---------+--------+--------------------+---------+--------------------+-------------+------------+-------------+--------------------+
|   1431.5|       1|       HL Road Frame|      680|HL Road Frame - B...|   FR-R92B-58|     1059.31|           14|{6461c642c7456de4...|
|   1431.5|       1|       HL Road Frame|      706|HL Road Frame - R...|   FR-R92R-58|     1059.31|           14|{6461c642c7456de4...|
|    34.99|       0|           Sport-100|      707|Sport-100 Helmet,...|    HL-U509-R|     13.0863|           31|{6461c642c7456de4...|
|    34.99|       0|           Sport-100|      708|Sport-100 Helmet,...|      HL-U509|     13.0863|           31|{6461c642c7456de4...|
|      9.5|       0| Mountain Bike Socks|      710|Moun

In [10]:
# spark.sql("CREATE TABLE ModelNames (ModelName string, ListPrice INT)")
spark.sql("CREATE TABLE ModelNames (ModelName string, ListPrice INT) USING parquet")

DataFrame[]

In [11]:
spark.sql("DESCRIBE EXTENDED ModelNames").show(100,100)

+----------------------------+----------------------------------------------------------+-------+
|                    col_name|                                                 data_type|comment|
+----------------------------+----------------------------------------------------------+-------+
|                   ModelName|                                                    string|   null|
|                   ListPrice|                                                       int|   null|
|                            |                                                          |       |
|# Detailed Table Information|                                                          |       |
|                    Database|                                                   default|       |
|                       Table|                                                modelnames|       |
|                Created Time|                              Mon May 15 06:14:44 UTC 2023|       |
|                 La

In [12]:
%ls -l spark-warehouse

total 4
drwxr-sr-x 2 jovyan users 4096 May 15 06:14 modelnames/


In [13]:
%ls -l spark-warehouse/modelnames

total 0


In [14]:
spark.sql("INSERT OVERWRITE TABLE ModelNames SELECT ModelName, ListPrice FROM external_products")

DataFrame[]

In [15]:
%ls -l spark-warehouse/modelnames

total 4
-rw-r--r-- 1 jovyan users 2859 May 15 06:14 part-00000-1e961db4-5922-4111-b644-32109d68373d-c000.snappy.parquet
-rw-r--r-- 1 jovyan users    0 May 15 06:14 _SUCCESS


In [16]:
spark.sql("SELECT * FROM ModelNames").show()

+--------------------+---------+
|           ModelName|ListPrice|
+--------------------+---------+
|       HL Road Frame|     1431|
|       HL Road Frame|     1431|
|           Sport-100|       34|
|           Sport-100|       34|
| Mountain Bike Socks|        9|
|           Sport-100|       34|
|         Cycling Cap|        8|
| Mountain Bike Socks|        9|
|Long-Sleeve Logo ...|       49|
|Long-Sleeve Logo ...|       49|
|Long-Sleeve Logo ...|       49|
|       HL Road Frame|     1431|
|       HL Road Frame|     1431|
|       HL Road Frame|     1431|
|       HL Road Frame|     1431|
|Long-Sleeve Logo ...|       49|
|       HL Road Frame|     1431|
|       LL Road Frame|      337|
|       LL Road Frame|      337|
|       LL Road Frame|      337|
+--------------------+---------+
only showing top 20 rows



In [17]:
spark.sql("DROP TABLE external_products")

DataFrame[]

In [18]:
spark.sql("SELECT * FROM ModelNames").show()

+--------------------+---------+
|           ModelName|ListPrice|
+--------------------+---------+
|       HL Road Frame|     1431|
|       HL Road Frame|     1431|
|           Sport-100|       34|
|           Sport-100|       34|
| Mountain Bike Socks|        9|
|           Sport-100|       34|
|         Cycling Cap|        8|
| Mountain Bike Socks|        9|
|Long-Sleeve Logo ...|       49|
|Long-Sleeve Logo ...|       49|
|Long-Sleeve Logo ...|       49|
|       HL Road Frame|     1431|
|       HL Road Frame|     1431|
|       HL Road Frame|     1431|
|       HL Road Frame|     1431|
|Long-Sleeve Logo ...|       49|
|       HL Road Frame|     1431|
|       LL Road Frame|      337|
|       LL Road Frame|      337|
|       LL Road Frame|      337|
+--------------------+---------+
only showing top 20 rows



In [19]:
%ls -l spark-warehouse/modelnames

total 4
-rw-r--r-- 1 jovyan users 2859 May 15 06:14 part-00000-1e961db4-5922-4111-b644-32109d68373d-c000.snappy.parquet
-rw-r--r-- 1 jovyan users    0 May 15 06:14 _SUCCESS


In [20]:
spark.sql("DROP TABLE ModelNames")

DataFrame[]

In [21]:
%ls -l spark-warehouse

total 0
